In [ ]:
# Import pyspark and set up SparkContext
import pyspark
sc = pyspark.SparkContext()

In [ ]:
# Create an rdd
rdd = sc.parallelize([(x, x%3) for x in range(10)], 5)
rdd.collect()

In [ ]:
# Flip the key-value pairs
flipped = rdd.map(lambda (x,y): (y,x))
flipped.collect()

In [ ]:
# Reduce
flipped.reduceByKey(lambda x,y: x+y).collect()

In [ ]:
# textFile
rdd2 = sc.textFile('link_text.txt')
rdd2.take(10)

In [ ]:
# retrieve site names
def get_site(iterator):
    for link in iterator:
        index = link.find("www.")
        end = link.find(".com")
        if index > 0 and end > 0:
            yield link[index + 4: end]

def f(link):
    index = link.find("www.")
    end = link.find(".com")
    if index > 0 and end > 0:
        return link[index + 4: end]

site_rdd = rdd2.mapPartitionsWithIndex(lambda index, iterator: get_site(iterator))
site_rdd.take(10)
# what's the difference?
site_rdd2 = rdd2.map(f)
site_rdd2.take(10)

In [ ]:
# implement distinct
class funRDD:
    rdd = None
    
    def __init__(self, rdd):
        self.rdd = rdd

    def distinct(self):
        """
        Return a new RDD containing the distinct elements in this RDD.

        >>> sorted(sc.parallelize([1, 1, 2, 3]).distinct().collect())
        [1, 2, 3]
        """
        return self.rdd.map(lambda x: (x, None)) \
                       .reduceByKey(lambda x, y: None) \
                       .map(lambda (x,y): x)

funRDD(site_rdd).distinct().collect() # retrieves distinct sites